# Libraries

In [1]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os
import datetime
import pandas as pd

# Token Creation

In [2]:
def create_token_file(credentials_path='../UserCredentials/credentials.json', token_path='../UserCredentials/token.json'):
    """
    Generate a token file for authenticating with the Google Calendar API.

    This function uses the OAuth 2.0 Installed Application flow to authenticate a user
    and create a token file that allows access to the Google Calendar API.

    Args:
        credentials_path (str): Path to the client secrets JSON file obtained from the Google Developer Console.
        token_path (str): Path to save the generated token file.

    Returns:
        None: The function saves the token to the specified path.
    
    Raises:
        FileNotFoundError: If the credentials file is not found at the given path.
        Exception: For any other errors during the token creation process.

    Example Usage:
        # Assuming you have your credentials JSON file at './credentials.json' and 
        # want to save the token at './token.json':

        create_token_file(
            credentials_path='./credentials.json',
            token_path='./token.json'
        )
    """
    # Scopes indicate what aspects of the user's Google account you want access to
    scopes = ['https://www.googleapis.com/auth/calendar']

    try:
        # Check if credentials file exists
        if not os.path.exists(credentials_path):
            raise FileNotFoundError(f"Credentials file not found at: {credentials_path}")
        
        # Create the flow using the client secrets file from the Google Developer Console
        flow = InstalledAppFlow.from_client_secrets_file(credentials_path, scopes=scopes)

        # Run the flow to generate the token
        creds = flow.run_local_server(port=0)

        # Ensure the directory for the token file exists
        os.makedirs(os.path.dirname(token_path), exist_ok=True)

        # Save the credentials to a token file
        with open(token_path, 'w') as token:
            token.write(creds.to_json())

        print(f"Token successfully stored in {token_path}")

    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


In [3]:
## usage
create_token_file() # for authentication

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=226204716140-mqn2q79c0t8vhlsdldmpt52d13kdku34.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52806%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=W1STwFY3P4rw0fxvDxnBQttzZa9uB0&access_type=offline
Token successfully stored in ../UserCredentials/token.json


# setup calendar service

In [4]:
def setup_calendar_service(credentials_path='../UserCredentials/credentials.json',
                           token_path='../UserCredentials/token.json'):
    """
    Set up and return a service object for accessing the Google Calendar API.

    This function ensures the user is authenticated with OAuth 2.0. If a valid token file exists,
    it uses it for authentication. Otherwise, it initiates the OAuth flow and creates a new token file.

    Args:
        credentials_path (str): Path to the OAuth 2.0 Client Credentials JSON file.
        token_path (str): Path to save the OAuth 2.0 tokens after the first successful authentication.

    Returns:
        googleapiclient.discovery.Resource: A service object for interacting with the Google Calendar API.

    Raises:
        FileNotFoundError: If the credentials file is not found at the specified path.
        Exception: For other unexpected errors during authentication or service creation.

    Example Usage:
        # Initialize the Google Calendar API service
        service = setup_calendar_service(
            credentials_path='./credentials.json',
            token_path='./token.json'
        )

        # Now you can use the `service` object to interact with the Calendar API
        calendar_list = service.calendarList().list().execute()
        print(calendar_list)
    """
    SCOPES = ['https://www.googleapis.com/auth/calendar']

    try:
        # Check if credentials file exists
        if not os.path.exists(credentials_path):
            raise FileNotFoundError(f"Credentials file not found at: {credentials_path}")

        # Attempt to load existing credentials from the token file
        if os.path.exists(token_path):
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
        else:
            # If the token file does not exist, initiate the OAuth flow
            flow = InstalledAppFlow.from_client_secrets_file(credentials_path, SCOPES)
            creds = flow.run_local_server(port=0)

            # Ensure the directory for the token file exists
            os.makedirs(os.path.dirname(token_path), exist_ok=True)

            # Save the new credentials to the token file
            with open(token_path, 'w') as token:
                token.write(creds.to_json())

        # Create the Google Calendar API service object using the credentials
        service = build('calendar', 'v3', credentials=creds)
        return service

    except FileNotFoundError as e:
        print(f"Error: {e}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise


In [5]:
service = setup_calendar_service()

# Global Dictinoary and Stack to store events


In [6]:
# Global dictionary and stack for tracking events
recent_keys_dic = {}  # Dictionary to store event keys and their corresponding event IDs
recent_keys_stack = []    # Stack to track the order of event creation (most recent on top)


# Create single event

need to change `create_single_event()` function again because it will create duplicate events.

In [32]:
def create_single_event(
    service,
    calendar_id='primary',
    title='title',
    description='description',
    event_date=None,
    event_time=None,
    end_date=None,
    timezone='America/Toronto',
    notifications=None,
    invitees=None,
):
    """
    Create a single event in Google Calendar and track it in global data structures.
    [docstring unchanged…]
    """
    # Use today's date as default for event_date and end_date if not provided
    event_date = event_date or datetime.date.today().strftime('%Y-%m-%d')
    end_date   = end_date   or event_date

    # Decide which reminders to use
    if notifications is None:
        if event_time:                 # timed (date‑time) events
            overrides = [
                {'method': 'email',  'minutes': 24 * 60},     # 1 day before (same clock time)
                {'method': 'popup',  'minutes': 7  * 24 * 60},# 1 week before
                {'method': 'popup',  'minutes': 2  * 60},     # 2 hours before
                {'method': 'popup',  'minutes': 24 * 60},     # 1 day before
                {'method': 'popup',  'minutes': 2  * 24 * 60} # 2 days before
            ]
        else:                           # all‑day events (start = 00:00)
            #H9 = 9 * 60                 # 9 a.m. offset
            overrides = [
                {'method': 'popup',  'minutes': 7  * 24 * 60}, # 1 week before @ 00:00
                {'method': 'email',  'minutes': 1  * 24 * 60}, # 1 day  before @ 00:00 (email)
                {'method': 'popup',  'minutes': 1  * 24 * 60}, # 1 day  before @ 00:00
                {'method': 'popup',  'minutes': 2  * 24 * 60},  # 2 days before @ 00:00
                {'method': 'popup',  'minutes': 3  * 24 * 60},  # 3 days before @ 00:00
            ]
    else:
        overrides = notifications


    # Generate a unique key for the event using its parameters
    key = '\u00B6'.join([
        '', title, description, event_date, event_time or '',
        end_date, timezone, ''
    ])

    if key in recent_keys_dic:
        return f"Event with key {key} already exists. No new event created."

    # Build the event object
    event = {
        'summary': title,
        'description': description,
        'start':  (
            {'dateTime': f"{event_date}T{event_time}", 'timeZone': timezone}
            if event_time else {'date': event_date}
        ),
        'end':    (
            {'dateTime': f"{end_date}T{event_time}", 'timeZone': timezone}
            if event_time else {'date': end_date}
        ),
        'reminders': {
            'useDefault': False,
            'overrides': overrides
        },
        'attendees': [{'email': e} for e in (invitees or [])]
    }

    created_event = service.events().insert(calendarId=calendar_id, body=event).execute()
    recent_keys_dic[key] = key
    recent_keys_stack.append(key)

    return f"Event created with ID: {key}"


## default - create single event

In [31]:
create_single_event(service, title = 'all day8 event')

'Event created with ID: ¶all day8 event¶description¶2025-05-06¶¶2025-05-06¶America/Toronto¶'

## create all day event (custom calendar)

In [82]:
create_single_event(
    service,
    calendar_id='6e3c8cd9ec14a3048ad48d83a107d3b6af2a17d67f4ca61340b43f46fbf77016@group.calendar.google.com',
    title='All-Day Event',
    description='An all-day meeting.',
)

'Event created with ID: ¶All-Day Event¶An all-day meeting.¶2025-01-24¶None¶2025-01-24¶America/Toronto¶'

## create a time specific event (with invitees)

In [83]:
create_single_event(
    service,
    title='Team Meeting',
    description='Discuss project progress.',
    event_date='2025-01-10',
    event_time='14:00',
    timezone='America/Toronto',
    notifications=[{'method': 'email', 'minutes': 30}, {'method': 'popup', 'minutes': 10}],
    invitees=['example1@gmail.com', 'example2@gmail.com'],
)


HttpError: <HttpError 400 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Bad Request". Details: "[{'domain': 'global', 'reason': 'badRequest', 'message': 'Bad Request'}]">

## Create a task (single due date)

In [9]:
create_single_event(
    service,
    title='Submit Report',
    description='Final deadline for report submission.',
    event_date='2025-05-06',  # Due date
    event_time=None,  # No time for tasks
    end_date='2025-05-06',  # Same as start
)

'Event created with ID: ¶Submit Report¶Final deadline for report submission.¶2025-05-06¶None¶2025-05-06¶America/Toronto¶'

# Which email

In [10]:
def get_authenticated_email(service):
    """
    Prints and returns the email address of the authenticated user.
    """
    try:
        calendar_list = service.calendarList().list().execute()
        primary_calendar = next(
            calendar for calendar in calendar_list['items'] if calendar.get('primary', False)
        )
        print(f"Authenticated email: {primary_calendar['id']}")
        return primary_calendar['id']
    except Exception as e:
        print(f"Error fetching authenticated email: {e}")



In [11]:

# Example usage:
get_authenticated_email(service)

Authenticated email: 03paulpark@gmail.com


'03paulpark@gmail.com'

# Delete new event

In [12]:
def delete_event_by_key(service, calendar_id='primary', key=None):
    """
    Deletes an event from Google Calendar using its key. If no key is provided, the most recently created event is deleted.
    The event is removed from both Google Calendar and local tracking structures.

    Args:
        service (googleapiclient.discovery.Resource): The Google Calendar API service object.
        calendar_id (str): The ID of the calendar. Defaults to 'primary'.
        key (str, optional): The unique key of the event to delete. Defaults to the most recent key.

    Returns:
        str: A message indicating success or failure.
    """
    global recent_keys_stack, recent_keys_dic

    try:
        # Determine the key to delete
        if key is None:
            if not recent_keys_stack:
                return "No events to delete."
            key = recent_keys_stack.pop()  # Use the most recent key from the stack

        # Debugging: Print the key being deleted
        print(f"Key to Delete: {key}")

        # Look up the eventId using the key
        if key in recent_keys_dic:
            event_id = recent_keys_dic[key]  # Get the event ID for the key

            try:
                # Delete the event from Google Calendar using eventId
                service.events().delete(calendarId=calendar_id, eventId=event_id).execute()
                print(f"Event with ID {event_id} deleted from Google Calendar.")

                # Remove the key from the dictionary
                del recent_keys_dic[key]
                print(f"Key {key} removed from recent_keys_dic.")
                return f"Deleted event with ID: {event_id} (key: {key})"

            except Exception as api_error:
                # Handle API-specific errors
                return f"Error deleting event from Google Calendar: {api_error}"

        else:
            return f"No event found with the specified key: {key}"

    except KeyError:
        return f"KeyError: Missing event data for key: {key}. Stack or dictionary may be out of sync."
    except Exception as e:
        return f"Unexpected error occurred: {e}"


In [13]:
# stack
recent_keys_dic

{'¶all day event¶description¶2025-05-06¶None¶2025-05-06¶America/Toronto¶': '¶all day event¶description¶2025-05-06¶None¶2025-05-06¶America/Toronto¶',
 '¶Submit Report¶Final deadline for report submission.¶2025-05-06¶None¶2025-05-06¶America/Toronto¶': '¶Submit Report¶Final deadline for report submission.¶2025-05-06¶None¶2025-05-06¶America/Toronto¶'}

In [14]:
# dictionary
recent_keys_stack

['¶all day event¶description¶2025-05-06¶None¶2025-05-06¶America/Toronto¶',
 '¶Submit Report¶Final deadline for report submission.¶2025-05-06¶None¶2025-05-06¶America/Toronto¶']

In [255]:
create_single_event(service)

'Event created with ID: ¶title¶description¶2025-01-24¶None¶2025-01-24¶America/Toronto¶'

In [186]:
service

In [248]:
calendar_id = '6e3c8cd9ec14a3048ad48d83a107d3b6af2a17d67f4ca61340b43f46fbf77016@group.calendar.google.com'  # found using your specific calendar and integrate settings
create_single_event(
    service=service,
    calendar_id=calendar_id,
    #title='Meeting',
    #description='Discuss project',
    #event_date='2025-01-25',
    #event_time='14:00',
    #timezone='America/Toronto',
    #notifications=[{'method':'email', 'minutes':5}],
    #invitees= []
)



'Event created with ID: ¶title¶description¶2025-01-24¶None¶2025-01-24¶America/Toronto¶'

In [239]:
create_single_event(service, title = 'all dayu event')

'Event created with ID: ¶all dayu event¶description¶2025-01-24¶None¶2025-01-24¶America/Toronto¶'

In [256]:
delete_event_by_key(service)

Key to Delete: ¶title¶description¶2025-01-24¶None¶2025-01-24¶America/Toronto¶


'Error deleting event from Google Calendar: <HttpError 404 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events/%C2%B6title%C2%B6description%C2%B62025-01-24%C2%B6None%C2%B62025-01-24%C2%B6America%2FToronto%C2%B6? returned "Not Found". Details: "[{\'domain\': \'global\', \'reason\': \'notFound\', \'message\': \'Not Found\'}]">'

In [18]:
delete_result = delete_event_by_key(service, key="all dayu event¶description¶2025-01-23¶None¶2025-01-23¶America/Toronto")
print(delete_result)


HttpError: <HttpError 410 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events/2a880sh87pc2p16kh1kgte61j0? returned "Resource has been deleted". Details: "[{'domain': 'global', 'reason': 'deleted', 'message': 'Resource has been deleted'}]">

# Handling events (create or delete)

In [15]:
def handle_events(service, calendar_id, events_df, operation, **kwargs):
    """
    Handles creating or deleting events based on the operation specified ('create' or 'delete').

    Args:
    service (Resource): The Google Calendar service object.
    calendar_id (str): The ID of the calendar where operations will be performed.
    events_df (pd.DataFrame): DataFrame containing event details.
    operation (str): Operation to perform - 'create' or 'delete'.
    **kwargs: Additional keyword arguments to pass to the event creation or deletion function.

    Returns:
    None
    """
    for _, row in events_df.iterrows():
        title = row['title']  # Corrected from 'Activity Description' to 'title'
        description = row.get('description', '')  # Ensure this matches your DataFrame column names
        event_date = row.get('event_date', None)  # Ensure this matches your DataFrame column names

        if operation == 'create':
            result = create_single_event(service, calendar_id=calendar_id, title=title,
                                         description=description, event_date=event_date, **kwargs)
            print(f"Event created: {result}")


In [47]:
import pandas as pd

# ------------------------  French course schedule  ------------------------ #
french_data = {
    "title": [
        # BEGIN entries  ▾
        "begin Test #1 : Chapitre 5",
        "begin Peer‑Review Teams Populated",
        "begin Test #2 : Chapitre 6",
        "begin Test #3 : Chapitre 7",
        "begin Test #4 : Chapitre 8",
        "begin Test #5 : Chapitre 9",
        "begin Final Test",
        # DUE / milestone entries ▾
        "Student Acknowledgement",
        "Test #1 : Chapitre 5",
        "Peer‑Review Teams Populated",
        "Written Assignment : Part A (Draft)",
        "Test #2 : Chapitre 6",
        "Written Assignment : Part B (Peer Review)",
        "Written Assignment : Part C (Final Draft)",
        "Test #3 : Chapitre 7",
        "Test #4 : Chapitre 8",
        "Oral Assignment (Chapitre 8)",
        "Test #5 : Chapitre 9",
        "Final Test"
    ],
    "event_date": [
        "2025-05-20",  # begin Test #1 (Tue)
        "2025-05-20",  # begin Peer‑Review Teams (Tue)
        "2025-06-02",  # begin Test #2  (Mon)
        "2025-06-16",  # begin Test #3  (Mon)
        "2025-07-02",  # begin Test #4  (Wed)
        "2025-07-14",  # begin Test #5  (Mon)
        "2025-07-21",  # begin Final Test (Mon)
        # due / milestone dates ▾
        "2025-05-12",  # Student Acknowledgement (Mon)
        "2025-05-23",  # Test #1 due (Fri)
        "2025-05-20",  # Teams populated (Tue, 16:30)
        "2025-05-30",  # Written A draft due (Fri)
        "2025-06-05",  # Test #2 due (Thu)
        "2025-06-06",  # Written B peer‑review due (Fri)
        "2025-06-13",  # Written C final draft due (Fri)
        "2025-06-19",  # Test #3 due (Thu)
        "2025-07-05",  # Test #4 due (Sat)
        "2025-07-11",  # Oral assignment due (Fri)
        "2025-07-17",  # Test #5 due (Thu)
        "2025-07-24"   # Final Test due (Thu)
    ],
    "description": [
        "Week 3,\nBegin = Tuesday, May 20, 2025: 11:55 PM",
        "Week 3,\nBegin = Tuesday, May 20, 2025: 4:30 PM",
        "Week 5,\nBegin = Monday, June 02, 2025: 11:55 PM",
        "Week 7,\nBegin = Monday, June 16, 2025: 11:55 PM",
        "Week 9,\nBegin = Wednesday, July 02, 2025: 11:55 PM",
        "Week 11,\nBegin = Monday, July 14, 2025: 11:55 PM",
        "Week 12,\nBegin = Monday, July 21, 2025: 11:55 PM",
        # due / milestone descriptions ▾
        "Week 2,\nDue Date = Monday, May 12, 2025: 11:55 PM",
        "Week 3,\nDue Date = Friday, May 23, 2025: 11:55 PM (Test #1)",
        "Week 3,\nDue Date = Tuesday, May 20, 2025: 4:30 PM (Teams populated)",
        "Week 4,\nDue Date = Friday, May 30, 2025: 11:55 PM",
        "Week 5,\nDue Date = Thursday, June 05, 2025: 11:55 PM (Test #2)",
        "Week 5,\nDue Date = Friday, June 06, 2025: 11:55 PM",
        "Week 6,\nDue Date = Friday, June 13, 2025: 11:55 PM",
        "Week 7,\nDue Date = Thursday, June 19, 2025: 11:55 PM (Test #3)",
        "Week 9,\nDue Date = Saturday, July 05, 2025: 11:55 PM (Test #4)",
        "Week 10,\nDue Date = Friday, July 11, 2025: 11:55 PM",
        "Week 11,\nDue Date = Thursday, July 17, 2025: 11:55 PM (Test #5)",
        "Week 12,\nDue Date = Thursday, July 24, 2025: 11:55 PM (Final Test)"
    ]
}
# Convert to DataFrame
french_due_dates_df = pd.DataFrame(french_data)


# ------------------------  PHIL 145 course schedule  ------------------------ #
phil145_data = {
    "title": [
        "Introduce Yourself",
        "Assignment 1 Available",
        "Assignment 1 Due",
        "Assignment 2 Available",
        "Assignment 2 Due",
        "Assignment 3 Available",
        "Assignment 3 Due",
        "Assignment 4 Available",
        "Assignment 4 Due",
        "Final Exam Available",
        "Final Exam Due"
    ],
    "event_date": [
        "2025-05-16",  # Fri
        "2025-05-14",  # Wed
        "2025-05-28",  # Wed
        "2025-06-04",  # Wed
        "2025-06-18",  # Wed
        "2025-06-25",  # Wed
        "2025-07-09",  # Wed
        "2025-07-11",  # Fri
        "2025-07-25",  # Fri
        "2025-07-28",  # Mon
        "2025-08-11"   # Mon
    ],
    "description": [
        "Week 1,\nDue Date = Friday, May 16, 2025: 11:55 PM (Ungraded)",
        "Week 2,\nAvailable = Wednesday, May 14, 2025",
        "Week 4,\nDue Date = Wednesday, May 28, 2025: 11:59 PM (17.5 %)",
        "Week 5,\nAvailable = Wednesday, June 04, 2025",
        "Week 7,\nDue Date = Wednesday, June 18, 2025: 11:59 PM (17.5 %)",
        "Week 8,\nAvailable = Wednesday, June 25, 2025",
        "Week 10,\nDue Date = Wednesday, July 09, 2025: 11:59 PM (17.5 %)",
        "Week 10,\nAvailable = Friday, July 11, 2025",
        "Week 12,\nDue Date = Friday, July 25, 2025: 11:59 PM (17.5 %)",
        "Final Exam,\nAvailable = Monday, July 28, 2025: 12:05 AM",
        "Final Exam,\nDue Date = Monday, August 11, 2025: 11:59 PM (30 %)"
    ]
}

# Convert to DataFrame

phil145_df = pd.DataFrame(phil145_data)

# ------------------------  Business‑Ethics schedule  ------------------------ #
phil215_data = {
    "title": [
        "Textbook Reading : Chapter 1",
        "Case Study Assignment 1 Available",
        "Case Study Assignment 1 Due",
        "Midterm Take‑home Examination Available",
        "Midterm Take‑home Examination Due",
        "Case Study Assignment 2 Available",
        "Case Study Assignment 2 Due",
        "Final Examination Available",
        "Final Examination Due"
    ],
    "event_date": [
        "2025-05-09",  # Fri
        "2025-05-23",  # Fri
        "2025-06-06",  # Fri
        "2025-06-18",  # Wed
        "2025-06-25",  # Wed
        "2025-07-02",  # Wed
        "2025-07-23",  # Wed
        "2025-07-28",  # Mon
        "2025-08-11"   # Mon
    ],
    "description": [
        "Week 1,\nDue Date = Friday, May 09, 2025: 11:55 PM",
        "Week 3,\nAvailable = Friday, May 23, 2025: 12:05 AM",
        "Week 5,\nDue Date = Friday, June 06, 2025: 11:55 PM (15 %)",
        "Week 7,\nAvailable = Wednesday, June 18, 2025: 12:05 AM",
        "Week 8,\nDue Date = Wednesday, June 25, 2025: 11:55 PM (25 %)",
        "Week 9,\nAvailable = Wednesday, July 02, 2025: 12:05 AM",
        "Week 12,\nDue Date = Wednesday, July 23, 2025: 11:55 PM (30 %)",
        "Final Exam,\nAvailable = Monday, July 28, 2025: 12:05 AM",
        "Final Exam,\nDue Date = Monday, August 11, 2025: 11:55 PM (30 %)"
    ]
}

# Convert to DataFrame
phil215_df = pd.DataFrame(phil215_data)

# Optional preview
print(phil215_df.head())

                                     title  event_date  \
0             Textbook Reading : Chapter 1  2025-05-09   
1        Case Study Assignment 1 Available  2025-05-23   
2              Case Study Assignment 1 Due  2025-06-06   
3  Midterm Take‑home Examination Available  2025-06-18   
4        Midterm Take‑home Examination Due  2025-06-25   

                                         description  
0  Week 1,\nDue Date = Friday, May 09, 2025: 11:5...  
1  Week 3,\nAvailable = Friday, May 23, 2025: 12:...  
2  Week 5,\nDue Date = Friday, June 06, 2025: 11:...  
3  Week 7,\nAvailable = Wednesday, June 18, 2025:...  
4  Week 8,\nDue Date = Wednesday, June 25, 2025: ...  


In [ ]:
## Calendar IDs
french = '2806f05d02c1bbdf138214309fef8f6c1066b1aa071308257a51ca49eedd2497@group.calendar.google.com'
phil145 = 'a8bdd0dc0a5fa4bf1b1ab1119d0fe386fad68b2d2828f887ad7863780b344e88@group.calendar.google.com'
phil215 = '0e428a6de7912c89056c8842895e4145c7d1e42c5c17db2475804361d2d94af7@group.calendar.google.com'

In [49]:
# Example usage
# Assuming 'service' is your Google Calendar service object
operation = 'create'  # or 'delete'
handle_events(service, calendar_id= french, events_df = french_due_dates_df, operation = operation)

Event created: Event created with ID: ¶begin Test #1 : Chapitre 5¶Week 3,
Begin = Tuesday, May 20, 2025: 11:55 PM¶2025-05-20¶¶2025-05-20¶America/Toronto¶
Event created: Event created with ID: ¶begin Peer‑Review Teams Populated¶Week 3,
Begin = Tuesday, May 20, 2025: 4:30 PM¶2025-05-20¶¶2025-05-20¶America/Toronto¶
Event created: Event created with ID: ¶begin Test #2 : Chapitre 6¶Week 5,
Begin = Monday, June 02, 2025: 11:55 PM¶2025-06-02¶¶2025-06-02¶America/Toronto¶
Event created: Event created with ID: ¶begin Test #3 : Chapitre 7¶Week 7,
Begin = Monday, June 16, 2025: 11:55 PM¶2025-06-16¶¶2025-06-16¶America/Toronto¶
Event created: Event created with ID: ¶begin Test #4 : Chapitre 8¶Week 9,
Begin = Wednesday, July 02, 2025: 11:55 PM¶2025-07-02¶¶2025-07-02¶America/Toronto¶
Event created: Event created with ID: ¶begin Test #5 : Chapitre 9¶Week 11,
Begin = Monday, July 14, 2025: 11:55 PM¶2025-07-14¶¶2025-07-14¶America/Toronto¶
Event created: Event created with ID: ¶begin Final Test¶Week 12,
B

In [131]:
def delete_all_events(service):
    """
    Deletes all events listed in the global `recent_keys_dic` dictionary from the specified Google Calendar.

    Args:
    service (Resource): The Google Calendar service object.

    Returns:
    None
    """
    # Copy keys to list to avoid RuntimeError due to change in dictionary size during iteration
    keys_to_delete = list(recent_keys_dic.keys())
    for key in keys_to_delete:
        result = delete_event_by_key(service, key)
        print(f"Event deleted with key {key}: {result}")

        # not working

need to fix delete operations, key is not working properly. Potential fix, create a key generation function and change create single event code.

# Main

In [ ]:
def main():
    service = setup_calendar_service()
    upcoming_events = fetch_upcoming_events(service)
    print("Upcoming Events:", upcoming_events)
    event_details = {"summary": "New Meeting", "time": "Tomorrow at 10 AM"}
    new_event = add_new_event(service, event_details)
    print("Added Event:", new_event)

# data

,title,event_date,description
0,Rights and Responsibilities Agreement,2025-01-10,"Week 1,\nDue Date = Friday, January 10, 2025: ..."
1,New Employee Orientation Exercise,2025-01-10,"Week 1,\nDue Date = Friday, January 10, 2025: ..."
